##Import Libraries

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

##Import Dataset

In [14]:
df = pd.read_csv('Fertilizer_Recommendation.csv')

##Encoder

In [15]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

##Defining X and target

In [17]:
X = df.drop(columns=["Recommended_Fertilizer"])
y = df["Recommended_Fertilizer"]
y_label_encoder = LabelEncoder()
y = y_label_encoder.fit_transform(y)

##Normalize Features

In [18]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##Model

In [20]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation= 'relu'),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


##Compile the model

In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4184 - loss: 1.6699 - val_accuracy: 0.7983 - val_loss: 0.7066
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7915 - loss: 0.6522 - val_accuracy: 0.8700 - val_loss: 0.3361
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8414 - loss: 0.4041 - val_accuracy: 0.9183 - val_loss: 0.2424
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8604 - loss: 0.3250 - val_accuracy: 0.9150 - val_loss: 0.2079
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8707 - loss: 0.3011 - val_accuracy: 0.9150 - val_loss: 0.1824
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8830 - loss: 0.2705 - val_accuracy: 0.9283 - val_loss: 0.1758
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9006 - loss: 0.2398 - val_accuracy: 0.9283 - val_loss: 0.1689
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8892 - loss: 0.2422 - val_accuracy: 0.9267 - val_loss:

In [28]:
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Accuracy: 0.955


In [29]:
model.save("fertilizer_mlp_model.h5")

In [30]:
def predict_fertilizer(input_data):
    input_data_scaled = scaler.transform([input_data])
    prediction = np.argmax(model.predict(input_data_scaled), axis=1)
    return y_label_encoder.inverse_transform(prediction)[0]

In [31]:
sample_input = [90, 42, 43, 20.87, 82.00, 6.50, 202.93, 2]  # Replace with real values
predicted_fertilizer = predict_fertilizer(sample_input)
print("Predicted Fertilizer:", predicted_fertilizer)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted Fertilizer: NPK Mix


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [33]:
import joblib

joblib.dump(model, 'fertilizer_mlp_model.pkl')

['fertilizer_mlp_model.pkl']